- 在上一讲中，我们展示了几种实现简易平均加权的方式
    - 在这里，做一些一些准备工作，从而实现后续搭建自注意力模块

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class BLM(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,idx,targets = None):

        logits = self.token_embedding_table(idx) # (B,T)  -> (B,T,C)  # 这里我们通过Embedding操作直接得到预测分数
        # 这里的预测分数过程与二分类或者多分类的分数是大致相同的

        
        if targets is None:
            loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)    # 这里我们调整一下形状，以符合torch的交叉熵损失函数对于输入的变量的要求
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        '''
        idx 是现在的输入的(B, T)序列 ，这是之前我们提取的batch的下标
        max_new_tokens 是产生的最大的tokens数量
        '''

        for _ in range(max_new_tokens):
            
            # 得到预测的结果
            logits,_ = self(idx) # _ 表示省略，用于不获取相对应的函数返回值
            
            # 只关注最后一个的预测  (B,T,C)
            logits = logits[:, -1, :] # becomes (B, C)
            # 对概率值应用softmax
            probs = F.softmax(logits, dim=-1) # (B, C)
            # nn.argmax
            # 对input的每一行做n_samples次取值，输出的张量是每一次取值时input张量对应行的下标，也即找到概率值输出最大的下标，也对应着最大的编码
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # 将新产生的编码加入到之前的编码中，形成新的编码
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx 